<a href="https://colab.research.google.com/github/SergioManuelJob/TicTacToe-Qlearning/blob/master/3enRaya_Sergio_Manuel_Su%C3%A1reaz_Su%C3%A1rez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Creamos tabla de transiciones

Nos ayudamos del metodo movimiento legal para quitarnos todos los estados ilegales.

In [ ]:
def movimientoLegal(array):

  contBlancas = 0
  contNegras = 0

  # 1 para negras 2 para blancas
  for i in range (0, 9, 1):
    if array[i] == 1:
      contNegras += 1
    if array[i] == 2:
      contBlancas += 1

  if contBlancas > contNegras:
    return False
  elif contNegras >= contBlancas + 2:
    return False
  else:
    return True

In [ ]:
# Vamos a hacer la tabla
estados = {}
indice = 0
for a in range (0, 3, 1):
  for b in range (0, 3, 1):
    for c in range (0, 3, 1):
      for d in range (0, 3, 1):
        for e in range (0, 3, 1):
          for f in range (0, 3, 1):
            for g in range (0, 3, 1):
              for h in range (0, 3, 1):
                for i in range (0, 3, 1):
                  if (movimientoLegal([a, b, c, d, e, f, g, h, i])):
                    estados[indice] =  a*6561 + b*2187 + c*729 + d*243 + e*81 + f*27 + g*9 + h*3 + i
                    indice += 1

In [ ]:
estados_inverso = {valor: clave for clave, valor in estados.items()}

In [ ]:
def crearTransiciones(arrayEstados):
    T = {}
    indice = 0
    for i in range(0,len(arrayEstados)):
        numero = arrayEstados[i]
        a, b, c, d, e, f, g, h, i = (numero // 6561) % 3, (numero // 2187) % 3, (numero // 729) % 3, (numero // 243) % 3, (numero // 81) % 3, (numero // 27) % 3, (numero // 9) % 3, (numero // 3) % 3, numero % 3
        T[indice] = [a, b, c, d, e, f, g, h, i]
        indice += 1
    return T

In [ ]:
T = crearTransiciones(estados)

### Creación de la tabla de recompensas

Para eso creamos metodos que devuelven un booleano de si ganas o empatas

In [ ]:
def ganar(array, numJugador):
    # Verificar filas
    for i in range(0, 9, 3):
        if array[i] == array[i+1] == array[i+2] and array[i] == numJugador:
            return True

    # Verificar columnas
    for i in range(3):
        if array[i] == array[i+3] == array[i+6] and array[i] == numJugador:
            return True

    # Verificar diagonales
    if array[0] == array[4] == array[8] and array[0] == numJugador:
        return True
    if array[2] == array[4] == array[6] and array[2] == numJugador:
        return True

    return False

In [ ]:
def tableroLleno(array):
  for i in array:
    if i == 0:
      return False
  else:
    return True

In [ ]:
R = {}
for i in T:
  # 1 para negras 2 para blancas
  if(ganar(T[i], 1)):
    R[i] = 100
  elif(ganar(T[i], 2)):
    R[i] = -100
  # elif(tableroLleno(T[i])):
  #   R[i] = 10
  else:
    R[i] = 0

### Creamos tabla Q

In [ ]:
Q = np.zeros((6046, 9))

### Entrenamos al agente de q-learning

Y nos ayudamos del metodo `calcularNumeroModular` para representar el tablero de forma modular.

In [ ]:
def calcularNumeroModular(array):
  return array[0] * 6561 + array[1] * 2187 + array[2] * 729 + array[3] * 243 + array[4] * 81 + array[5] * 27 + array[6] * 9 + array[7] * 3 + array[8]

In [ ]:
v = 0.1
y = 0.9

max_entrenamientos = 100000
s = 0
entrenar = 0

while entrenar < max_entrenamientos:

    a = np.random.randint(9)

    while T[s][a] != 0:
      a = np.random.randint(9)

    # Se coloca la ficha
    array = T[s].copy()
    array[a] = 1
    siguiente = calcularNumeroModular(array)
    indiceSiguiente = estados_inverso[siguiente]

    # Se cambia la tabla Q
    Q[s, a] = (1 - v) * Q[s, a] + v * (R[s+1] + y * np.max(Q[indiceSiguiente, :]))

    # Se actualiza el estado
    s = indiceSiguiente

    # Se comprueba si el juego ha terminado y se reinicia
    if ganar(T[s], 1) or tableroLleno(T[s]):
        s = 0

    # Se comprueba que el jugador 2 nunca juegue primero para evitar casos ilegales
    if entrenar < max_entrenamientos and s != 0:

        a_player2 = np.random.randint(9)

        while T[s][a_player2] != 0:
          a_player2 = np.random.randint(9)

        # Se coloca la ficha del jugador 2
        array = T[s].copy()
        array[a_player2] = 2
        siguiente_player2 = calcularNumeroModular(array)
        indiceSiguiente_player2 = estados_inverso[siguiente_player2]

        # Se actualiza el estado para el jugador 2
        s = indiceSiguiente_player2

        # Se comprueba si el juego ha terminado para el jugador 2
        if ganar(T[s], 2) or tableroLleno(T[s]):
            s = 0


    entrenar += 1

### Revisión de la tabla Q para ver cómo ha entrenado.

In [ ]:
T[1]

[0, 0, 0, 0, 0, 0, 0, 0, 1]

In [ ]:
def imprimir_matriz_q(Q):
    for i, fila in enumerate(Q):
        for j, valor in enumerate(fila):
            print(f"Q[{i}, {j}] = {valor:.2f}")

imprimir_matriz_q(Q)

In [ ]:
np.argmax(Q[5966, :])

6

### Metodo para jugar contra un agente random

In [ ]:
max_entrenamientos = 1000
victorias = 0
empates = 0
derrotas = 0

for _ in range(max_entrenamientos):
    s = 0
    turno_agente = True

    while True:
        if turno_agente:
            # Agente Q-learning
            a = np.argmax(Q[s, :])
        else:
            # Agente aleatorio
            a = np.random.randint(9)

        while T[s][a] != 0:
            if turno_agente:
                a = np.random.randint(9)
            else:
                a = np.random.randint(9)

        # Se coloca la ficha
        array = T[s].copy()
        array[a] = 1 if turno_agente else 2
        siguiente = calcularNumeroModular(array)
        indice_siguiente = estados_inverso[siguiente]

        # Se actualiza el estado
        s = indice_siguiente

        # Se comprueba si el juego ha terminado y se actualizan las estadísticas
        if ganar(T[s], 1) or ganar(T[s], 2) or tableroLleno(T[s]):
            if ganar(T[s], 1):
                victorias += 1
            elif ganar(T[s], 2):
                derrotas += 1
            else:
                empates += 1
            turno_agente = True
            s = 0
            break

        turno_agente = not turno_agente

print(f"Resultados después de {max_entrenamientos} partidas:")
print(f"Victorias: {victorias}")
print(f"Empates: {empates}")
print(f"Derrotas: {derrotas}")


Resultados después de 1000 partidas:
Victorias: 627
Empates: 117
Derrotas: 256


### Jugar contra un agente experto

Que es el movimiento optimo, hay varios ifs que controlan primero que no haya movimientos ganadores (para si el enemigo va a ganar impedirlo, y si él puede ganar, hacerlo), después de tomar el centro si no está tomado, y después una esquina o sino un lado.

In [ ]:
def movimiento_optimo(tablero):
    # Buscar movimientos ganadores o bloquear al oponente
    arrayAux = tablero.copy()
    for i in range(3):
        for j in range(3):
            if arrayAux[i * 3 + j] == 0:
                arrayAux[i * 3 + j] = 2
                if ganar(arrayAux, 2):
                    return i * 3 + j
                arrayAux[i * 3 + j] = 0

    # Evitar perder en el siguiente movimiento del jugador
    for i in range(3):
        for j in range(3):
            if arrayAux[i * 3 + j] == 0:
                arrayAux[i * 3 + j] = 1
                if ganar(arrayAux, 1):
                    return i * 3 + j
                arrayAux[i * 3 + j] = 0

    # Si no hay movimientos ganadores, tomar el centro si está disponible
    if arrayAux[4] == 0:
        return 4

    # Si no, tomar una esquina si está disponible
    for pos in [0, 2, 6, 8]:
        if arrayAux[pos] == 0:
            return pos

    # Si no, tomar un lado
    for pos in [1, 3, 5, 7]:
        if arrayAux[pos] == 0:
            return pos

### Metodo de jugar contra el agente experto

In [ ]:
max_entrenamientos = 1000
victorias = 0
empates = 0
derrotas = 0

for _ in range(max_entrenamientos):
    s = 0
    turno_agente = True

    while True:
        if turno_agente:

            a = np.argmax(Q[s, :])
        else:
            a = movimiento_optimo(T[s])

        while T[s][a] != 0:
            if turno_agente:
                a = np.random.randint(9)
            else:
               a = movimiento_optimo(T[s])

        # Se coloca la ficha
        array = T[s].copy()
        array[a] = 1 if turno_agente else 2
        siguiente = calcularNumeroModular(array)
        indice_siguiente = estados_inverso[siguiente]
        # Se actualiza el estado
        s = indice_siguiente

        # Se comprueba si el juego ha terminado y se actualizan las estadísticas
        if ganar(T[s], 1) or ganar(T[s], 2) or tableroLleno(T[s]):
            if ganar(T[s], 1):
                victorias += 1
            elif ganar(T[s], 2):
                derrotas += 1
            else:
                empates += 1
            turno_agente = True
            s = 0
            break

        turno_agente = not turno_agente

print(f"Resultados después de {max_entrenamientos} partidas:")
print(f"Victorias: {victorias}")
print(f"Empates: {empates}")
print(f"Derrotas: {derrotas}")

Resultados después de 1000 partidas:
Victorias: 32
Empates: 137
Derrotas: 831
